0 - Preparation

In [33]:
# Install required 

# !pip install scikit-learn

In [34]:
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB

In [35]:
# Load the dataset

# Specify the path to the wiki_data.csv file
data_path = "Data/wiki_data.csv"

# Load the data into a Pandas DataFrame
df = pd.read_csv(data_path)

In [36]:
df.head()

,id,url,title,wiki_intro,generated_intro,title_len,wiki_intro_len,generated_intro_len,prompt,generated_text,prompt_tokens,generated_text_tokens
0,63064638,https://en.wikipedia.org/wiki/Sexhow%20railway...,Sexhow railway station,Sexhow railway station was a railway station b...,Sexhow railway station was a railway station l...,3,174,78,200 word wikipedia style introduction on 'Sexh...,"located in the town of Sexhow, on the Cumbria...",25,88
1,279621,https://en.wikipedia.org/wiki/Eti%C3%A4inen,Etiäinen,"In Finnish folklore, all places and things, an...","In Finnish folklore, all places and things, an...",1,187,80,200 word wikipedia style introduction on 'Etiä...,"animate or inanimate, have a spirit or ""etiäi...",26,101
2,287229,https://en.wikipedia.org/wiki/Inverse%20functi...,Inverse function theorem,"In mathematics, specifically differential calc...","In mathematics, specifically differential calc...",3,170,59,200 word wikipedia style introduction on 'Inve...,function theorem states that for every real-v...,26,65
3,26712375,https://en.wikipedia.org/wiki/Stepping%20on%20...,Stepping on Roses,is a Japanese shōjo manga series written and i...,is a Japanese shōjo manga series written and i...,3,335,121,200 word wikipedia style introduction on 'Step...,and illustrated by Maki Fujii. The series fol...,26,150
4,38894426,https://en.wikipedia.org/wiki/Rob%20Bradley,Rob Bradley,"Robert Milner ""Rob"" Bradley, Jr. (born August ...","Robert Milner ""Rob"" Bradley, Jr. (born August ...",2,170,136,200 word wikipedia style introduction on 'Rob ...,"29, 1973) is an American former professional ...",28,162


In [37]:
# Use only a fraction of the data for faster training iterations
fraction = 0.5  # Adjust this to use, e.g., 1% of the data

# Calculate the number of samples to include for each class
num_human_samples = int(len(df) * fraction)
num_ai_samples = int(len(df) * fraction)

# Create a balanced dataset with an equal number of human and AI-generated samples
balanced_texts = df['wiki_intro'].tolist()[:num_human_samples] + df['generated_intro'].tolist()[:num_ai_samples]
balanced_labels = [0] * num_human_samples + [1] * num_ai_samples

# Split the balanced dataset into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(balanced_texts, balanced_labels, test_size=0.1)

1 - TF-IDF - Logistic Classifier

In [38]:
# Use TF-IDF to transform text to numerical feature
vectorizer = TfidfVectorizer(max_features=5000)  # Adjust max_features based on your dataset
X_train = vectorizer.fit_transform(train_texts)
X_val = vectorizer.transform(val_texts)

In [39]:
# Model Training
log_model = LogisticRegression()
log_model = log_model.fit(X_train, train_labels)

C:\Users\soere\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [40]:
# Model Evaluation
val_predictions = log_model.predict(X_val)
accuracy = accuracy_score(val_labels, val_predictions)
print("Validation Accuracy:", accuracy)
print("\nClassification Report:\n", classification_report(val_labels, val_predictions))

Validation Accuracy: 0.9333333333333333

Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.93      0.93      7460
           1       0.93      0.94      0.93      7540

    accuracy                           0.93     15000
   macro avg       0.93      0.93      0.93     15000
weighted avg       0.93      0.93      0.93     15000



In [41]:
# Use TF-IDF to transform the entire dataset's text to numerical features
vectorizer = TfidfVectorizer(max_features=5000)  # Adjust max_features based on your dataset
X = vectorizer.fit_transform(balanced_texts)
y = balanced_labels

# Cross-Validation
cv_log_model = LogisticRegression()
cross_val_scores = cross_val_score(cv_log_model, X, y, cv=5, scoring='accuracy')

print("Cross-Validation Accuracy Scores:", cross_val_scores)
print("Mean CV Accuracy:", cross_val_scores.mean())

C:\Users\soere\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\soere\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mo

Cross-Validation Accuracy Scores: [0.93406667 0.93213333 0.93256667 0.9316     0.93313333]
Mean CV Accuracy: 0.9327


C:\Users\soere\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


1 - TF-IDF - Naive Bayes

In [45]:
# Initialize the Multinomial Naive Bayes model
nb_model = MultinomialNB()

# Train the model using the training data
nb_model.fit(X_train, train_labels)

# Make predictions on the validation set
val_predictions_nb = nb_model.predict(X_val)

# Calculate and print the accuracy
accuracy_nb = accuracy_score(val_labels, val_predictions_nb)
print("Naive Bayes Validation Accuracy:", accuracy_nb)

# Optional: Detailed classification report
print("\nNaive Bayes Classification Report:\n", classification_report(val_labels, val_predictions_nb))


Naive Bayes Validation Accuracy: 0.8064

Naive Bayes Classification Report:
               precision    recall  f1-score   support

           0       0.78      0.85      0.81      7460
           1       0.84      0.76      0.80      7540

    accuracy                           0.81     15000
   macro avg       0.81      0.81      0.81     15000
weighted avg       0.81      0.81      0.81     15000



In [46]:
# Initialize the Multinomial Naive Bayes model
nb_model = MultinomialNB()

In [47]:
# Perform Cross-Validation
nb_cross_val_scores = cross_val_score(nb_model, X, y, cv=5, scoring='accuracy')

print("Naive Bayes Cross-Validation Accuracy Scores:", nb_cross_val_scores)
print("Naive Bayes Mean CV Accuracy:", nb_cross_val_scores.mean())

# Optional: Train the model on the entire dataset if satisfied with CV results
nb_model.fit(X, y)

Naive Bayes Cross-Validation Accuracy Scores: [0.81236667 0.80513333 0.80346667 0.80473333 0.8035    ]
Naive Bayes Mean CV Accuracy: 0.8058399999999999


MultinomialNB()